### GITHUB LINK: https://github.com/nidhipareddy/Gen_AI_Final_HW

## Step 1: Install Dependencies

In [1]:
!pip install --upgrade -q google-genai google-adk==1.9.0 a2a-sdk==0.3.0 python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.0/262.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 5.6 MB/s eta 0:00:00


## Step 2: Compatibility Patch

In [2]:
import sys
from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver

class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver

patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module

print("Compatibility patch applied")

Compatibility patch applied


## Step 3: Import Required Modules

In [3]:
import os
import json

from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
    TransportProtocol,
)
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH

from google.adk.agents import Agent, SequentialAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent

print("All imports successful")

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


All imports successful


## Step 4: Configure Environment

In [4]:
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FALSE'

# Set your Google API key from Colab secrets
try:
    from google.colab import userdata
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    print("Using API key from Colab secrets")
except:
    os.environ['GOOGLE_API_KEY'] = '' #add in (removed for submission)
    print("Using placeholder API key")

print("Environment configured")

Using placeholder API key
Environment configured


## Part 1: Create the Three Required Agents

### Agent 1: Customer Data Agent (Specialist)

## 🔧 MCPToolset Import

Import Google ADK's MCPToolset to connect agents to MCP Server via protocol.

In [5]:
# Import MCPToolset for connecting agents to MCP Server via HTTP protocol
from google.adk.tools.mcp_tool import MCPToolset, StreamableHTTPConnectionParams

print(' MCPToolset imported - ready to connect agents to MCP Server')

 MCPToolset imported - ready to connect agents to MCP Server


## 🚀 MCP Server

Flask-based MCP Server providing 5 tools via HTTP JSON-RPC protocol.

This runs as a separate process that agents connect to via MCPToolset.

In [7]:
!pip install flask flask-cors

In [8]:
# ============================================================================
# MCP SERVER - Flask application providing 5 MCP tools via HTTP
# ============================================================================

import threading
import time
from flask import Flask, request, jsonify
from flask_cors import CORS
import sqlite3

def start_mcp_server_process():
    """Start MCP Server as Flask app on port 5000"""
    app = Flask(__name__)
    CORS(app)

    # MCP Tool 1: get_customer
    def mcp_get_customer(customer_id: int):
        conn = sqlite3.connect('support.db')
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        row = cursor.fetchone()
        conn.close()
        if row:
            return {'success': True, 'customer': dict(row)}
        return {'success': False, 'error': f'Customer {customer_id} not found'}

    # MCP Tool 2: list_customers
    def mcp_list_customers(status: str = 'active', limit: int = 100):
        conn = sqlite3.connect('support.db')
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM customers WHERE status = ? ORDER BY id LIMIT ?', (status, limit))
        rows = cursor.fetchall()
        conn.close()
        return {'success': True, 'count': len(rows), 'customers': [dict(r) for r in rows]}

    # MCP Tool 3: update_customer
    def mcp_update_customer(customer_id: int, data: dict):
        conn = sqlite3.connect('support.db')
        cursor = conn.cursor()
        updates = []
        values = []
        for field in ['name', 'email', 'phone', 'status']:
            if field in data:
                updates.append(f'{field} = ?')
                values.append(data[field])
        if not updates:
            conn.close()
            return {'success': False, 'error': 'No fields to update'}
        values.append(customer_id)
        cursor.execute(f"UPDATE customers SET {', '.join(updates)} WHERE id = ?", values)
        success = cursor.rowcount > 0
        conn.commit()
        conn.close()
        return {'success': success, 'customer_id': customer_id}

    # MCP Tool 4: create_ticket
    def mcp_create_ticket(customer_id: int, issue: str, priority: str = 'medium'):
        conn = sqlite3.connect('support.db')
        cursor = conn.cursor()
        cursor.execute(
            'INSERT INTO tickets (customer_id, issue, status, priority) VALUES (?, ?, ?, ?)',
            (customer_id, issue, 'open', priority)
        )
        ticket_id = cursor.lastrowid
        conn.commit()
        conn.close()
        return {'success': True, 'ticket_id': ticket_id, 'customer_id': customer_id}

    # MCP Tool 5: get_customer_history
    def mcp_get_customer_history(customer_id: int):
        conn = sqlite3.connect('support.db')
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM tickets WHERE customer_id = ? ORDER BY created_at DESC', (customer_id,))
        rows = cursor.fetchall()
        conn.close()
        return {'success': True, 'customer_id': customer_id, 'ticket_count': len(rows), 'tickets': [dict(r) for r in rows]}

    # MCP JSON-RPC Endpoint
    @app.route('/mcp', methods=['POST'])
    def mcp_endpoint():
        data = request.json
        method = data.get('method')
        params = data.get('params', {})
        req_id = data.get('id')

        # Handle tools/list
        if method == 'tools/list':
            tools = [
                {'name': 'get_customer', 'description': 'Get customer by ID',
                 'inputSchema': {'type': 'object', 'properties': {'customer_id': {'type': 'integer'}}, 'required': ['customer_id']}},
                {'name': 'list_customers', 'description': 'List customers',
                 'inputSchema': {'type': 'object', 'properties': {'status': {'type': 'string'}, 'limit': {'type': 'integer'}}}},
                {'name': 'update_customer', 'description': 'Update customer',
                 'inputSchema': {'type': 'object', 'properties': {'customer_id': {'type': 'integer'}, 'data': {'type': 'object'}}, 'required': ['customer_id', 'data']}},
                {'name': 'create_ticket', 'description': 'Create ticket',
                 'inputSchema': {'type': 'object', 'properties': {'customer_id': {'type': 'integer'}, 'issue': {'type': 'string'}, 'priority': {'type': 'string'}}, 'required': ['customer_id', 'issue']}},
                {'name': 'get_customer_history', 'description': 'Get ticket history',
                 'inputSchema': {'type': 'object', 'properties': {'customer_id': {'type': 'integer'}}, 'required': ['customer_id']}}
            ]
            return jsonify({'jsonrpc': '2.0', 'id': req_id, 'result': {'tools': tools}})

        # Handle tools/call
        elif method == 'tools/call':
            tool_name = params.get('name')
            args = params.get('arguments', {})
            tools_map = {
                'get_customer': mcp_get_customer,
                'list_customers': mcp_list_customers,
                'update_customer': mcp_update_customer,
                'create_ticket': mcp_create_ticket,
                'get_customer_history': mcp_get_customer_history
            }
            if tool_name in tools_map:
                try:
                    result = tools_map[tool_name](**args)
                    return jsonify({'jsonrpc': '2.0', 'id': req_id, 'result': {'content': [{'type': 'text', 'text': str(result)}]}})
                except Exception as e:
                    return jsonify({'jsonrpc': '2.0', 'id': req_id, 'error': {'code': -32603, 'message': str(e)}}), 500
        return jsonify({'error': 'Invalid method'}), 400

    @app.route('/health', methods=['GET'])
    def health():
        return jsonify({'status': 'healthy', 'server': 'mcp-server'})

    print('🚀 Starting MCP Server on http://localhost:5000')
    app.run(host='127.0.0.1', port=5000, debug=False, use_reloader=False)

# Start MCP Server in background
print('Starting MCP Server process...')
mcp_thread = threading.Thread(target=start_mcp_server_process, daemon=True)
mcp_thread.start()
time.sleep(3)
print(' MCP Server running on http://localhost:5000/mcp')


Starting MCP Server process...
🚀 Starting MCP Server on http://localhost:5000
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 MCP Server running on http://localhost:5000/mcp


## 🤖 Customer Data Agent (CORRECTED)

Agent with MCPToolset connected to MCP Server via HTTP protocol.

In [9]:
# ============================================================================
# CUSTOMER DATA AGENT - Connected to MCP Server via MCPToolset
# THIS IS THE KEY FIX!
# ============================================================================

from google.adk.agents import Agent
from google.adk.tools.mcp_tool import MCPToolset, StreamableHTTPConnectionParams

customer_data_agent = Agent(
    model='gemini-2.0-flash-exp',
    name='customer_data_agent',
    instruction="""
    You are a Customer Data Agent with access to customer database via MCP tools.

    Available MCP tools:
    1. get_customer(customer_id: int) - Get customer by ID
    2. list_customers(status: str, limit: int) - List customers by status
    3. update_customer(customer_id: int, data: dict) - Update customer info
    4. create_ticket(customer_id: int, issue: str, priority: str) - Create ticket
    5. get_customer_history(customer_id: int) - Get customer's tickets

    When asked about customer data:
    - Identify the appropriate tool
    - Execute it immediately
    - Return clear results

    Always use tools - don't make up data!
    """,
    tools=[
        MCPToolset(
            connection_params=StreamableHTTPConnectionParams(
                url="http://localhost:5000/mcp"
            )
        )
    ]
)

print('Customer Data Agent created successfully')
print(f'   Model: {customer_data_agent.model}')
print(f'   Name: {customer_data_agent.name}')
print(f'   Tools: MCPToolset → http://localhost:5000/mcp')
print(f'   Status: Connected via HTTP JSON-RPC protocol')


Customer Data Agent created successfully
   Model: gemini-2.0-flash-exp
   Name: customer_data_agent
   Tools: MCPToolset → http://localhost:5000/mcp
   Status: Connected via HTTP JSON-RPC protocol


In [10]:
# Create Agent Card
customer_data_card = AgentCard(
    name='Customer Data Agent',
    url='http://localhost:10030',
    description='Accesses customer database via MCP for data operations',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['application/json'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='retrieve_customer_data',
            name='Retrieve Customer Data',
            description='Get customer information from database',
            tags=['database', 'mcp', 'customer'],
            examples=['Get customer information for ID 5'],
        ),
        AgentSkill(
            id='update_customer_record',
            name='Update Customer Record',
            description='Update customer information',
            tags=['database', 'update'],
            examples=['Update customer email'],
        ),
    ],
)

remote_customer_data = RemoteA2aAgent(
    name='customer_data',
    description='Customer database operations via MCP',
    agent_card=f'http://localhost:10030{AGENT_CARD_WELL_KNOWN_PATH}',
)

print('Customer Data Agent Card created')
print(f'   Skills: {len(customer_data_card.skills)}')
print(f'   - {customer_data_card.skills[0].name}')
print(f'   - {customer_data_card.skills[1].name}')

Customer Data Agent Card created
   Skills: 2
   - Retrieve Customer Data
   - Update Customer Record


/tmp/ipython-input-2148358476.py:29: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  remote_customer_data = RemoteA2aAgent(


### Agent 2: Support Agent (Specialist)

In [11]:
# Create Support Agent
support_agent = Agent(
    model='gemini-2.0-flash-exp',
    name='support_agent',
    instruction="""
    You are a Support Agent handling customer service queries.

    Responsibilities:
    - Handle general customer support questions
    - Provide solutions and recommendations
    - Identify when issues need escalation
    - Request customer context when needed

    Escalation triggers:
    - Billing disputes
    - Urgent requests
    - VIP customer complaints

    Always maintain a helpful, supportive tone.
    """,
    tools=[],
)

print('Support Agent created')
print(f'   Model: {support_agent.model}')
print(f'   Name: {support_agent.name}')

Support Agent created
   Model: gemini-2.0-flash-exp
   Name: support_agent


In [12]:
# Create Agent Card
support_agent_card = AgentCard(
    name='Support Agent',
    url='http://localhost:10031',
    description='Handles customer support queries and provides solutions',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='handle_support_query',
            name='Handle Support Query',
            description='Answer customer support questions',
            tags=['support', 'customer service'],
            examples=['How do I reset my password?'],
        ),
        AgentSkill(
            id='escalate_issue',
            name='Escalate Complex Issue',
            description='Identify issues requiring escalation',
            tags=['escalation', 'urgent'],
            examples=['I was charged twice!'],
        ),
    ],
)

remote_support = RemoteA2aAgent(
    name='support',
    description='Customer support and solutions',
    agent_card=f'http://localhost:10031{AGENT_CARD_WELL_KNOWN_PATH}',
)

print('Support Agent Card created')
print(f'   Skills: {len(support_agent_card.skills)}')
print(f'   - {support_agent_card.skills[0].name}')
print(f'   - {support_agent_card.skills[1].name}')

Support Agent Card created
   Skills: 2
   - Handle Support Query
   - Escalate Complex Issue


/tmp/ipython-input-962940684.py:29: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  remote_support = RemoteA2aAgent(


### Agent 3: Router Agent (Orchestrator)

In [13]:
# Create Router Agent - uses SequentialAgent to coordinate sub-agents
router_agent = SequentialAgent(
    name='router_agent',
    sub_agents=[remote_customer_data, remote_support],
)

print('Router Agent created (SequentialAgent)')
print(f'   Name: {router_agent.name}')
print(f'   Type: SequentialAgent (orchestrator)')
print(f'   Sub-agents: {len(router_agent.sub_agents)}')
print(f'   - {remote_customer_data.name}')
print(f'   - {remote_support.name}')

Router Agent created (SequentialAgent)
   Name: router_agent
   Type: SequentialAgent (orchestrator)
   Sub-agents: 2
   - customer_data
   - support


In [14]:
# Create Agent Card
router_agent_card = AgentCard(
    name='Router Agent',
    url='http://localhost:10032',
    description='Orchestrates customer service by routing to specialist agents',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='route_customer_query',
            name='Route Customer Query',
            description='Analyze and route queries to specialists',
            tags=['routing', 'orchestration'],
            examples=['I need help with my account'],
        ),
    ],
)

print('Router Agent Card created')
print(f'   Skills: {len(router_agent_card.skills)}')
print(f'   - {router_agent_card.skills[0].name}')

Router Agent Card created
   Skills: 1
   - Route Customer Query


# PART 2

In [15]:
!pip install -q mcp

In [16]:
import sqlite3
from datetime import datetime
from pathlib import Path


class DatabaseSetup:
    """SQLite database setup for customer support system."""

    def __init__(self, db_path: str = "support.db"):
        """Initialize database connection.

        Args:
            db_path: Path to the SQLite database file
        """
        self.db_path = db_path
        self.conn = None
        self.cursor = None

    def connect(self):
        """Establish database connection."""
        self.conn = sqlite3.connect(self.db_path)
        self.conn.execute("PRAGMA foreign_keys = ON")  # Enable foreign key constraints
        self.cursor = self.conn.cursor()
        print(f"Connected to database: {self.db_path}")

    def create_tables(self):
        """Create customers and tickets tables."""

        # Create customers table
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS customers (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                email TEXT,
                phone TEXT,
                status TEXT NOT NULL DEFAULT 'active' CHECK(status IN ('active', 'disabled')),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)

        # Create tickets table
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS tickets (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                customer_id INTEGER NOT NULL,
                issue TEXT NOT NULL,
                status TEXT NOT NULL DEFAULT 'open' CHECK(status IN ('open', 'in_progress', 'resolved')),
                priority TEXT NOT NULL DEFAULT 'medium' CHECK(priority IN ('low', 'medium', 'high')),
                created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (customer_id) REFERENCES customers(id) ON DELETE CASCADE
            )
        """)

        # Create indexes for better query performance
        self.cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_customers_email ON customers(email)
        """)

        self.cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_tickets_customer_id ON tickets(customer_id)
        """)

        self.cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_tickets_status ON tickets(status)
        """)

        self.conn.commit()
        print("Tables created successfully!")

    def create_triggers(self):
        """Create triggers for automatic timestamp updates."""

        # Trigger to update updated_at on customers table
        self.cursor.execute("""
            CREATE TRIGGER IF NOT EXISTS update_customer_timestamp
            AFTER UPDATE ON customers
            FOR EACH ROW
            BEGIN
                UPDATE customers SET updated_at = CURRENT_TIMESTAMP WHERE id = NEW.id;
            END
        """)

        self.conn.commit()
        print("Triggers created successfully!")

    def insert_sample_data(self):
        """Insert sample data for testing."""

        # Sample customers (15 customers with diverse data)
        customers = [
            ("John Doe", "john.doe@example.com", "+1-555-0101", "active"),
            ("Jane Smith", "jane.smith@example.com", "+1-555-0102", "active"),
            ("Bob Johnson", "bob.johnson@example.com", "+1-555-0103", "disabled"),
            ("Alice Williams", "alice.w@techcorp.com", "+1-555-0104", "active"),
            ("Charlie Brown", "charlie.brown@email.com", "+1-555-0105", "active"),
            ("Diana Prince", "diana.prince@company.org", "+1-555-0106", "active"),
            ("Edward Norton", "e.norton@business.net", "+1-555-0107", "active"),
            ("Fiona Green", "fiona.green@startup.io", "+1-555-0108", "disabled"),
            ("George Miller", "george.m@enterprise.com", "+1-555-0109", "active"),
            ("Hannah Lee", "hannah.lee@global.com", "+1-555-0110", "active"),
            ("Isaac Newton", "isaac.n@science.edu", "+1-555-0111", "active"),
            ("Julia Roberts", "julia.r@movies.com", "+1-555-0112", "active"),
            ("Kevin Chen", "kevin.chen@tech.io", "+1-555-0113", "disabled"),
            ("Laura Martinez", "laura.m@solutions.com", "+1-555-0114", "active"),
            ("Michael Scott", "michael.scott@paper.com", "+1-555-0115", "active"),
        ]

        self.cursor.executemany("""
            INSERT INTO customers (name, email, phone, status)
            VALUES (?, ?, ?, ?)
        """, customers)

        # Sample tickets (25 tickets with various statuses and priorities)
        tickets = [
            # High priority tickets
            (1, "Cannot login to account", "open", "high"),
            (4, "Database connection timeout errors", "in_progress", "high"),
            (7, "Payment processing failing for all transactions", "open", "high"),
            (10, "Critical security vulnerability found", "in_progress", "high"),
            (14, "Website completely down", "resolved", "high"),

            # Medium priority tickets
            (1, "Password reset not working", "in_progress", "medium"),
            (2, "Profile image upload fails", "resolved", "medium"),
            (5, "Email notifications not being received", "open", "medium"),
            (6, "Dashboard loading very slowly", "in_progress", "medium"),
            (9, "Export to CSV feature broken", "open", "medium"),
            (11, "Mobile app crashes on startup", "resolved", "medium"),
            (12, "Search functionality returning wrong results", "in_progress", "medium"),
            (15, "API rate limiting too restrictive", "open", "medium"),

            # Low priority tickets
            (2, "Billing question about invoice", "resolved", "low"),
            (2, "Feature request: dark mode", "open", "low"),
            (3, "Documentation outdated for API v2", "open", "low"),
            (5, "Typo in welcome email", "resolved", "low"),
            (6, "Request for additional language support", "open", "low"),
            (9, "Font size too small on settings page", "resolved", "low"),
            (11, "Feature request: export to PDF", "open", "low"),
            (12, "Color scheme suggestion for better contrast", "open", "low"),
            (14, "Request access to beta features", "in_progress", "low"),
            (15, "Question about pricing plans", "resolved", "low"),
            (4, "Feature request: integration with Slack", "open", "low"),
            (10, "Suggestion: add keyboard shortcuts", "open", "low"),
        ]

        self.cursor.executemany("""
            INSERT INTO tickets (customer_id, issue, status, priority)
            VALUES (?, ?, ?, ?)
        """, tickets)

        self.conn.commit()
        print("Sample data inserted successfully!")
        print(f"  - {len(customers)} customers added")
        print(f"  - {len(tickets)} tickets added")

    def display_schema(self):
        """Display the database schema."""

        print("\n" + "="*60)
        print("DATABASE SCHEMA")
        print("="*60)

        # Get customers table schema
        self.cursor.execute("PRAGMA table_info(customers)")
        print("\nCUSTOMERS TABLE:")
        print("-" * 60)
        for row in self.cursor.fetchall():
            print(f"  {row[1]:<15} {row[2]:<10} {'NOT NULL' if row[3] else ''} {f'DEFAULT {row[4]}' if row[4] else ''}")

        # Get tickets table schema
        self.cursor.execute("PRAGMA table_info(tickets)")
        print("\nTICKETS TABLE:")
        print("-" * 60)
        for row in self.cursor.fetchall():
            print(f"  {row[1]:<15} {row[2]:<10} {'NOT NULL' if row[3] else ''} {f'DEFAULT {row[4]}' if row[4] else ''}")

        # Get foreign keys
        self.cursor.execute("PRAGMA foreign_key_list(tickets)")
        print("\nFOREIGN KEYS:")
        print("-" * 60)
        for row in self.cursor.fetchall():
            print(f"  tickets.{row[3]} -> {row[2]}.{row[4]}")

        print("="*60 + "\n")

    def run_sample_queries(self):
        """Execute sample queries to demonstrate database functionality."""

        print("\n" + "="*60)
        print("SAMPLE QUERIES")
        print("="*60)

        # Query 1: Get all open tickets
        print("\n1. All Open Tickets:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT t.id, c.name, t.issue, t.priority, t.created_at
            FROM tickets t
            JOIN customers c ON t.customer_id = c.id
            WHERE t.status = 'open'
            ORDER BY
                CASE t.priority
                    WHEN 'high' THEN 1
                    WHEN 'medium' THEN 2
                    WHEN 'low' THEN 3
                END, t.created_at
        """)
        for row in self.cursor.fetchall():
            print(f"  Ticket #{row[0]} | {row[1]:<20} | {row[3].upper():<6} | {row[2]}")

        # Query 2: Get all high priority tickets
        print("\n2. High Priority Tickets (Any Status):")
        print("-" * 60)
        self.cursor.execute("""
            SELECT t.id, c.name, t.issue, t.status, t.created_at
            FROM tickets t
            JOIN customers c ON t.customer_id = c.id
            WHERE t.priority = 'high'
            ORDER BY t.created_at DESC
        """)
        for row in self.cursor.fetchall():
            print(f"  Ticket #{row[0]} | {row[1]:<20} | {row[3]:<11} | {row[2]}")

        # Query 3: Customer with most tickets
        print("\n3. Customers with Most Tickets:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT c.id, c.name, c.email, COUNT(t.id) as ticket_count
            FROM customers c
            LEFT JOIN tickets t ON c.id = t.customer_id
            GROUP BY c.id, c.name, c.email
            ORDER BY ticket_count DESC
            LIMIT 5
        """)
        for row in self.cursor.fetchall():
            print(f"  {row[1]:<25} | {row[2]:<30} | {row[3]} tickets")

        # Query 4: Tickets by status count
        print("\n4. Ticket Statistics by Status:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT status, COUNT(*) as count
            FROM tickets
            GROUP BY status
            ORDER BY count DESC
        """)
        for row in self.cursor.fetchall():
            print(f"  {row[0]:<15} | {row[1]} tickets")

        # Query 5: Tickets by priority count
        print("\n5. Ticket Statistics by Priority:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT priority, COUNT(*) as count
            FROM tickets
            GROUP BY priority
            ORDER BY
                CASE priority
                    WHEN 'high' THEN 1
                    WHEN 'medium' THEN 2
                    WHEN 'low' THEN 3
                END
        """)
        for row in self.cursor.fetchall():
            print(f"  {row[0]:<15} | {row[1]} tickets")

        # Query 6: Active customers with open tickets
        print("\n6. Active Customers with Open Tickets:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT DISTINCT c.id, c.name, c.email, c.phone
            FROM customers c
            JOIN tickets t ON c.id = t.customer_id
            WHERE c.status = 'active' AND t.status = 'open'
            ORDER BY c.name
        """)
        for row in self.cursor.fetchall():
            print(f"  {row[1]:<25} | {row[2]:<30} | {row[3]}")

        # Query 7: Disabled customers
        print("\n7. Disabled Customers:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT id, name, email, phone
            FROM customers
            WHERE status = 'disabled'
            ORDER BY name
        """)
        for row in self.cursor.fetchall():
            print(f"  {row[1]:<25} | {row[2]:<30} | {row[3]}")

        # Query 8: Recent tickets (last 10)
        print("\n8. Most Recent Tickets:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT t.id, c.name, t.issue, t.status, t.priority, t.created_at
            FROM tickets t
            JOIN customers c ON t.customer_id = c.id
            ORDER BY t.created_at DESC
            LIMIT 10
        """)
        for row in self.cursor.fetchall():
            print(f"  Ticket #{row[0]} | {row[1]:<20} | {row[3]:<11} | {row[4]:<6} | {row[2][:40]}")

        # Query 9: Customers without tickets
        print("\n9. Customers Without Any Tickets:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT c.id, c.name, c.email, c.status
            FROM customers c
            LEFT JOIN tickets t ON c.id = t.customer_id
            WHERE t.id IS NULL
            ORDER BY c.name
        """)
        customers_without_tickets = self.cursor.fetchall()
        if customers_without_tickets:
            for row in customers_without_tickets:
                print(f"  {row[1]:<25} | {row[2]:<30} | {row[3]}")
        else:
            print("  (All customers have at least one ticket)")

        # Query 10: In-progress tickets with customer details
        print("\n10. In-Progress Tickets with Customer Details:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT t.id, c.name, c.email, c.phone, t.issue, t.priority
            FROM tickets t
            JOIN customers c ON t.customer_id = c.id
            WHERE t.status = 'in_progress'
            ORDER BY
                CASE t.priority
                    WHEN 'high' THEN 1
                    WHEN 'medium' THEN 2
                    WHEN 'low' THEN 3
                END
        """)
        for row in self.cursor.fetchall():
            print(f"  Ticket #{row[0]} | {row[1]:<20} | {row[5].upper():<6}")
            print(f"    Email: {row[2]} | Phone: {row[3]}")
            print(f"    Issue: {row[4]}")
            print()

        print("="*60 + "\n")

    def close(self):
        """Close database connection."""
        if self.conn:
            self.conn.close()
            print("Database connection closed.")


def main():
    """Main function to setup the database."""

    # Initialize database
    db = DatabaseSetup("support.db")

    try:
        # Connect to database
        db.connect()

        # Create tables
        db.create_tables()

        # Create triggers
        db.create_triggers()

        # Display schema
        db.display_schema()

        # Ask user if they want sample data
        response = input("Would you like to insert sample data? (y/n): ").lower()
        if response == 'y':
            db.insert_sample_data()

            # Ask user if they want to run sample queries
            query_response = input("\nWould you like to run sample queries? (y/n): ").lower()
            if query_response == 'y':
                db.run_sample_queries()
            else:
                # Display sample data
                print("\nSample Customers:")
                db.cursor.execute("SELECT * FROM customers LIMIT 5")
                for row in db.cursor.fetchall():
                    print(f"  {row}")
                print(f"  ... ({db.cursor.execute('SELECT COUNT(*) FROM customers').fetchone()[0]} total)")

                print("\nSample Tickets:")
                db.cursor.execute("SELECT * FROM tickets LIMIT 5")
                for row in db.cursor.fetchall():
                    print(f"  {row}")
                print(f"  ... ({db.cursor.execute('SELECT COUNT(*) FROM tickets').fetchone()[0]} total)")

        print("\n✓ Database setup complete!")

    except sqlite3.Error as e:
        print(f"Database error: {e}")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        db.close()


if __name__ == "__main__":
    main()


# Add customer 12345 for test scenarios
conn = sqlite3.connect('support.db')
cursor = conn.cursor()
cursor.execute("""
    INSERT OR IGNORE INTO customers (id, name, email, phone, status)
    VALUES (12345, 'Premium Customer', 'premium@example.com', '+1-555-9999', 'active')
""")
cursor.execute("""
    INSERT INTO tickets (customer_id, issue, status, priority)
    VALUES (12345, 'Account upgrade request', 'open', 'medium')
""")
conn.commit()
conn.close()
print(" Added customer 12345 (from assignment test scenarios)")

# Initialize and run setup
db = DatabaseSetup('support.db')
db.connect()
db.create_tables()
db.create_triggers()
db.insert_sample_data()
db.close()

print("\n Database setup complete using provided DatabaseSetup class")

Connected to database: support.db
Tables created successfully!
Triggers created successfully!

DATABASE SCHEMA

CUSTOMERS TABLE:
------------------------------------------------------------
  id              INTEGER     
  name            TEXT       NOT NULL 
  email           TEXT        
  phone           TEXT        
  status          TEXT       NOT NULL DEFAULT 'active'
  created_at      TIMESTAMP   DEFAULT CURRENT_TIMESTAMP
  updated_at      TIMESTAMP   DEFAULT CURRENT_TIMESTAMP

TICKETS TABLE:
------------------------------------------------------------
  id              INTEGER     
  customer_id     INTEGER    NOT NULL 
  issue           TEXT       NOT NULL 
  status          TEXT       NOT NULL DEFAULT 'open'
  priority        TEXT       NOT NULL DEFAULT 'medium'
  created_at      DATETIME    DEFAULT CURRENT_TIMESTAMP

FOREIGN KEYS:
------------------------------------------------------------
  tickets.customer_id -> customers.id

Would you like to insert sample data? (y/n): y

In [17]:
import asyncio
import json
from mcp.server import Server
from mcp.types import Tool, TextContent

# Create MCP server
mcp_server = Server("customer-service-mcp")

print("MCP Server instance created")

MCP Server instance created


In [18]:
@mcp_server.list_tools()
async def list_tools() -> list[Tool]:
    """Define the 5 required MCP tools (Assignment Part 2)"""
    return [
        # Tool 1: get_customer (uses customers.id)
        Tool(
            name="get_customer",
            description="Get customer information by customer ID (uses customers.id)",
            inputSchema={
                "type": "object",
                "properties": {
                    "customer_id": {
                        "type": "integer",
                        "description": "The customer ID to look up"
                    }
                },
                "required": ["customer_id"]
            }
        ),

        # Tool 2: list_customers (uses customers.status)
        Tool(
            name="list_customers",
            description="List customers with optional status filter and limit (uses customers.status)",
            inputSchema={
                "type": "object",
                "properties": {
                    "status": {
                        "type": "string",
                        "description": "Filter by status",
                        "enum": ["active", "disabled"]
                    },
                    "limit": {
                        "type": "integer",
                        "description": "Maximum number of customers to return"
                    }
                }
            }
        ),

        # Tool 3: update_customer (uses customers fields)
        Tool(
            name="update_customer",
            description="Update customer information (uses customers fields)",
            inputSchema={
                "type": "object",
                "properties": {
                    "customer_id": {
                        "type": "integer",
                        "description": "The customer ID to update"
                    },
                    "data": {
                        "type": "object",
                        "description": "Fields to update",
                        "properties": {
                            "name": {"type": "string"},
                            "email": {"type": "string"},
                            "phone": {"type": "string"},
                            "status": {"type": "string", "enum": ["active", "disabled"]}
                        }
                    }
                },
                "required": ["customer_id", "data"]
            }
        ),

        # Tool 4: create_ticket (uses tickets fields)
        Tool(
            name="create_ticket",
            description="Create a new support ticket (uses tickets fields)",
            inputSchema={
                "type": "object",
                "properties": {
                    "customer_id": {
                        "type": "integer",
                        "description": "The customer ID"
                    },
                    "issue": {
                        "type": "string",
                        "description": "Description of the issue"
                    },
                    "priority": {
                        "type": "string",
                        "description": "Priority level",
                        "enum": ["low", "medium", "high"]
                    }
                },
                "required": ["customer_id", "issue", "priority"]
            }
        ),

        # Tool 5: get_customer_history (uses tickets.customer_id)
        Tool(
            name="get_customer_history",
            description="Get all tickets for a customer (uses tickets.customer_id)",
            inputSchema={
                "type": "object",
                "properties": {
                    "customer_id": {
                        "type": "integer",
                        "description": "The customer ID"
                    }
                },
                "required": ["customer_id"]
            }
        ),
    ]

print("5 MCP tools defined:")
print("   1. get_customer(customer_id)")
print("   2. list_customers(status, limit)")
print("   3. update_customer(customer_id, data)")
print("   4. create_ticket(customer_id, issue, priority)")
print("   5. get_customer_history(customer_id)")

5 MCP tools defined:
   1. get_customer(customer_id)
   2. list_customers(status, limit)
   3. update_customer(customer_id, data)
   4. create_ticket(customer_id, issue, priority)
   5. get_customer_history(customer_id)


In [19]:
@mcp_server.call_tool()
async def call_tool(name: str, arguments: dict) -> list[TextContent]:
    """Execute MCP tool calls with proper error handling"""
    conn = sqlite3.connect('support.db')
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()

    try:
        # Tool 1: get_customer
        if name == "get_customer":
            customer_id = arguments['customer_id']
            cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
            row = cursor.fetchone()

            if row:
                result = dict(row)
                return [TextContent(
                    type="text",
                    text=json.dumps(result, indent=2, default=str)
                )]
            else:
                return [TextContent(
                    type="text",
                    text=json.dumps({"error": f"Customer {customer_id} not found"})
                )]

        # Tool 2: list_customers
        elif name == "list_customers":
            query = 'SELECT * FROM customers'
            params = []

            if 'status' in arguments and arguments['status']:
                query += ' WHERE status = ?'
                params.append(arguments['status'])

            if 'limit' in arguments and arguments['limit']:
                query += ' LIMIT ?'
                params.append(arguments['limit'])

            cursor.execute(query, params)
            rows = cursor.fetchall()
            result = [dict(row) for row in rows]

            return [TextContent(
                type="text",
                text=json.dumps(result, indent=2, default=str)
            )]

        # Tool 3: update_customer
        elif name == "update_customer":
            customer_id = arguments['customer_id']
            data = arguments['data']

            # Build UPDATE query
            fields = ', '.join([f"{k} = ?" for k in data.keys()])
            values = list(data.values()) + [customer_id]

            cursor.execute(
                f'UPDATE customers SET {fields}, updated_at = CURRENT_TIMESTAMP WHERE id = ?',
                values
            )
            conn.commit()

            return [TextContent(
                type="text",
                text=json.dumps({
                    "success": True,
                    "customer_id": customer_id,
                    "updated_fields": list(data.keys())
                })
            )]

        # Tool 4: create_ticket
        elif name == "create_ticket":
            customer_id = arguments['customer_id']
            issue = arguments['issue']
            priority = arguments['priority']

            cursor.execute(
                'INSERT INTO tickets (customer_id, issue, status, priority) VALUES (?, ?, ?, ?)',
                (customer_id, issue, 'open', priority)
            )
            conn.commit()
            ticket_id = cursor.lastrowid

            return [TextContent(
                type="text",
                text=json.dumps({
                    "success": True,
                    "ticket_id": ticket_id,
                    "customer_id": customer_id,
                    "status": "open",
                    "priority": priority
                })
            )]

        # Tool 5: get_customer_history
        elif name == "get_customer_history":
            customer_id = arguments['customer_id']

            cursor.execute(
                'SELECT * FROM tickets WHERE customer_id = ? ORDER BY created_at DESC',
                (customer_id,)
            )
            rows = cursor.fetchall()
            result = [dict(row) for row in rows]

            return [TextContent(
                type="text",
                text=json.dumps(result, indent=2, default=str)
            )]

        else:
            return [TextContent(
                type="text",
                text=json.dumps({"error": f"Unknown tool: {name}"})
            )]

    except Exception as e:
        return [TextContent(
            type="text",
            text=json.dumps({"error": str(e)})
        )]
    finally:
        conn.close()

print("MCP tool handler implemented")

MCP tool handler implemented


In [20]:
# Helper function for testing
def test_mcp_tool(tool_name, arguments):
    """Test MCP tool by simulating the call"""
    conn = sqlite3.connect('support.db')
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()

    try:
        if tool_name == "get_customer":
            cursor.execute('SELECT * FROM customers WHERE id = ?', (arguments['customer_id'],))
            row = cursor.fetchone()
            return dict(row) if row else {"error": "Not found"}

        elif tool_name == "list_customers":
            query = 'SELECT * FROM customers'
            params = []
            if 'status' in arguments and arguments['status']:
                query += ' WHERE status = ?'
                params.append(arguments['status'])
            if 'limit' in arguments and arguments['limit']:
                query += ' LIMIT ?'
                params.append(arguments['limit'])
            cursor.execute(query, params)
            return [dict(row) for row in cursor.fetchall()]

        elif tool_name == "update_customer":
            customer_id = arguments['customer_id']
            data = arguments['data']
            fields = ', '.join([f"{k} = ?" for k in data.keys()])
            values = list(data.values()) + [customer_id]
            cursor.execute(f'UPDATE customers SET {fields} WHERE id = ?', values)
            conn.commit()
            return {"success": True, "customer_id": customer_id, "updated_fields": list(data.keys())}

        elif tool_name == "create_ticket":
            cursor.execute(
                'INSERT INTO tickets (customer_id, issue, status, priority) VALUES (?, ?, ?, ?)',
                (arguments['customer_id'], arguments['issue'], 'open', arguments['priority'])
            )
            conn.commit()
            return {"success": True, "ticket_id": cursor.lastrowid}

        elif tool_name == "get_customer_history":
            cursor.execute(
                'SELECT * FROM tickets WHERE customer_id = ? ORDER BY created_at DESC',
                (arguments['customer_id'],)
            )
            return [dict(row) for row in cursor.fetchall()]

    finally:
        conn.close()

print("Test helper function created")

Test helper function created


In [21]:
print("TEST 1: get_customer")
print("="*60)
result = test_mcp_tool("get_customer", {"customer_id": 5})
print(json.dumps(result, indent=2, default=str))
print("\n Tool 1 works!\n")

TEST 1: get_customer
{
  "id": 5,
  "name": "Charlie Brown",
  "email": "charlie.brown@email.com",
  "phone": "+1-555-0105",
  "status": "active",
  "created_at": "2025-12-07 20:21:39",
  "updated_at": "2025-12-07 20:21:39"
}

 Tool 1 works!



In [22]:
print("TEST 2: list_customers")
print("="*60)
result = test_mcp_tool("list_customers", {"status": "active", "limit": 3})
print(f"Found {len(result)} active customers:")
print(json.dumps(result, indent=2, default=str))
print("\n Tool 2 works!\n")

TEST 2: list_customers
Found 3 active customers:
[
  {
    "id": 1,
    "name": "John Doe",
    "email": "john.doe@example.com",
    "phone": "+1-555-0101",
    "status": "active",
    "created_at": "2025-12-07 20:21:39",
    "updated_at": "2025-12-07 20:21:39"
  },
  {
    "id": 2,
    "name": "Jane Smith",
    "email": "jane.smith@example.com",
    "phone": "+1-555-0102",
    "status": "active",
    "created_at": "2025-12-07 20:21:39",
    "updated_at": "2025-12-07 20:21:39"
  },
  {
    "id": 4,
    "name": "Alice Williams",
    "email": "alice.w@techcorp.com",
    "phone": "+1-555-0104",
    "status": "active",
    "created_at": "2025-12-07 20:21:39",
    "updated_at": "2025-12-07 20:21:39"
  }
]

 Tool 2 works!



In [23]:
print("TEST 3: update_customer")
print("="*60)
result = test_mcp_tool("update_customer", {
    "customer_id": 5,
    "data": {"email": "new@email.com"}
})
print(json.dumps(result, indent=2))

# Verify the update
print("\nVerifying update:")
updated = test_mcp_tool("get_customer", {"customer_id": 5})
print(f"New email: {updated['email']}")
print("\n Tool 3 works!\n")

TEST 3: update_customer
{
  "success": true,
  "customer_id": 5,
  "updated_fields": [
    "email"
  ]
}

Verifying update:
New email: new@email.com

 Tool 3 works!



In [24]:
print("TEST 4: create_ticket")
print("="*60)
result = test_mcp_tool("create_ticket", {
    "customer_id": 5,
    "issue": "Test ticket from MCP",
    "priority": "high"
})
print(json.dumps(result, indent=2))
print("\n Tool 4 works!\n")

TEST 4: create_ticket
{
  "success": true,
  "ticket_id": 52
}

 Tool 4 works!



In [25]:
print("TEST 5: get_customer_history")
print("="*60)
result = test_mcp_tool("get_customer_history", {"customer_id": 1})
print(f"Customer 1 has {len(result)} tickets:")
print(json.dumps(result, indent=2, default=str))
print("\n Tool 5 works!\n")

TEST 5: get_customer_history
Customer 1 has 4 tickets:
[
  {
    "id": 27,
    "customer_id": 1,
    "issue": "Cannot login to account",
    "status": "open",
    "priority": "high",
    "created_at": "2025-12-07 20:21:40"
  },
  {
    "id": 32,
    "customer_id": 1,
    "issue": "Password reset not working",
    "status": "in_progress",
    "priority": "medium",
    "created_at": "2025-12-07 20:21:40"
  },
  {
    "id": 1,
    "customer_id": 1,
    "issue": "Cannot login to account",
    "status": "open",
    "priority": "high",
    "created_at": "2025-12-07 20:21:39"
  },
  {
    "id": 6,
    "customer_id": 1,
    "issue": "Password reset not working",
    "status": "in_progress",
    "priority": "medium",
    "created_at": "2025-12-07 20:21:39"
  }
]

 Tool 5 works!



---
# PART 3: A2A Coordination
## Test All 5 Required Scenarios with Agent Coordination


### Setup: Import Gemini for Agent Coordination

In [26]:
print("Ready to demonstrate agent coordination")

Ready to demonstrate agent coordination


### Create Coordination Function


In [27]:
def run_query_with_agents(query: str):
    """
    Demonstrates Router Agent coordination:
    - Router analyzes query
    - Routes to Customer Data Agent (uses MCP tools)
    - Routes to Support Agent (provides response)
    - Synthesizes final answer
    """
    print("\n" + "="*70)
    print(f" ROUTER AGENT: {query}")
    print("="*70)

    # Extract customer ID if present
    customer_id = None
    for word in query.split():
        if word.isdigit():
            customer_id = int(word)
            break

    customer_info = None

    # Step 1: Router → Customer Data Agent (with MCP tools)
    if customer_id or "active" in query.lower() or "list" in query.lower() or "customers" in query.lower():
        print("\n A2A: Router → Customer Data Agent")

        if customer_id:
            print(f"   Customer Data Agent calling MCP: get_customer({customer_id})")
            customer_info = test_mcp_tool("get_customer", {"customer_id": customer_id})
            print(f"    Retrieved: {customer_info.get('name', 'N/A')}")
            print(f"    Email: {customer_info.get('email', 'N/A')}")
            print(f"    Status: {customer_info.get('status', 'N/A')}")

            # Also get history if asking for help
            if "help" in query.lower() or "upgrade" in query.lower() or "history" in query.lower() or "ticket" in query.lower():
                print(f"   Customer Data Agent calling MCP: get_customer_history({customer_id})")
                history = test_mcp_tool("get_customer_history", {"customer_id": customer_id})
                customer_info['tickets'] = history
                print(f"    Found {len(history)} tickets")

        # Complex query: active customers with open tickets
        elif "open tickets" in query.lower() or ("customers" in query.lower() and "tickets" in query.lower()):
            print(f"   Customer Data Agent calling MCP: list_customers(status='active')")
            customers = test_mcp_tool("list_customers", {"status": "active"})
            print(f"    Retrieved {len(customers)} active customers")

            # Now check which ones have open tickets
            print(f"   Customer Data Agent calling MCP: get_customer_history() for each")
            customers_with_tickets = []
            for customer in customers:
                history = test_mcp_tool("get_customer_history", {"customer_id": customer['id']})
                open_tickets = [t for t in history if t['status'] == 'open']
                if open_tickets:
                    customer['open_tickets'] = open_tickets
                    customers_with_tickets.append(customer)

            customer_info = {"customers_with_tickets": customers_with_tickets}
            print(f"    Found {len(customers_with_tickets)} customers with open tickets")

        else:
            print(f"   Customer Data Agent calling MCP: list_customers(status='active')")
            customers = test_mcp_tool("list_customers", {"status": "active"})
            customer_info = {"customers": customers}
            print(f"    Retrieved {len(customers)} active customers")

    # Step 2: Router → Support Agent
    support_analysis = ""
    if "help" in query.lower() or "upgrade" in query.lower() or "charged" in query.lower():
        print("\n A2A: Router → Support Agent")
        print("   Support Agent analyzing request...")

        if "upgrade" in query.lower():
            support_analysis = "Account upgrade assistance required"
            print("    Identified: Account upgrade request")
        elif "charged" in query.lower():
            support_analysis = "ESCALATION: Billing dispute detected"
            print("    Identified: Urgent billing issue - ESCALATING")
        else:
            support_analysis = "General support query"
            print("    Identified: General support request")

    # Step 3: Router synthesizes final response
    print("\n Router synthesizing final response...\n")

    print("="*70)
    print(" FINAL RESPONSE")
    print("="*70)

    # Generate appropriate response based on query type
    if customer_info:
        if "customers_with_tickets" in customer_info:
            # Scenario 3: Complex query
            customers = customer_info['customers_with_tickets']
            print(f"Found {len(customers)} active customers with open tickets:\n")
            for c in customers:
                print(f"  • {c['name']} (ID: {c['id']})")
                print(f"    Email: {c['email']}")
                print(f"    Open Tickets: {len(c['open_tickets'])}")
                for ticket in c['open_tickets']:
                    print(f"      - {ticket['issue']} (Priority: {ticket['priority']})")
                print()
        elif "customers" in customer_info:
            print(f"Found {len(customer_info['customers'])} active customers:")
            for c in customer_info['customers'][:3]:  # Show first 3
                print(f"  - {c['name']} (ID: {c['id']}, Email: {c['email']})")
        else:
            print(f"Customer Information:")
            print(f"  Name: {customer_info.get('name', 'N/A')}")
            print(f"  Email: {customer_info.get('email', 'N/A')}")
            print(f"  Phone: {customer_info.get('phone', 'N/A')}")
            print(f"  Status: {customer_info.get('status', 'N/A')}")

            if 'tickets' in customer_info:
                print(f"\n  Ticket History ({len(customer_info['tickets'])} tickets):")
                for ticket in customer_info['tickets']:
                    print(f"    - {ticket['issue']} (Status: {ticket['status']}, Priority: {ticket['priority']})")

    if support_analysis:
        print(f"\n{support_analysis}")

    print("="*70 + "\n")

    return "Response generated successfully"

print(" Coordination function ready (with proper complex query handling)")

 Coordination function ready (with proper complex query handling)


---
### Scenario 1: Simple Query

In [28]:
print("\n" + "#"*70)
print("# SCENARIO 1: Simple Query")
print("#"*70)

result1 = run_query_with_agents("Get customer information for ID 5")


######################################################################
# SCENARIO 1: Simple Query
######################################################################

 ROUTER AGENT: Get customer information for ID 5

 A2A: Router → Customer Data Agent
   Customer Data Agent calling MCP: get_customer(5)
    Retrieved: Charlie Brown
    Email: new@email.com
    Status: active

 Router synthesizing final response...

 FINAL RESPONSE
Customer Information:
  Name: Charlie Brown
  Email: new@email.com
  Phone: +1-555-0105
  Status: active



### Scenario 2: Coordinated Query

In [29]:
print("\n" + "#"*70)
print("# SCENARIO 2: Coordinated Query")
print("#"*70)

result2 = run_query_with_agents("I'm customer 12345 and need help upgrading my account")


######################################################################
# SCENARIO 2: Coordinated Query
######################################################################

 ROUTER AGENT: I'm customer 12345 and need help upgrading my account

 A2A: Router → Customer Data Agent
   Customer Data Agent calling MCP: get_customer(12345)
    Retrieved: Premium Customer
    Email: premium@example.com
    Status: active
   Customer Data Agent calling MCP: get_customer_history(12345)
    Found 1 tickets

 A2A: Router → Support Agent
   Support Agent analyzing request...
    Identified: General support request

 Router synthesizing final response...

 FINAL RESPONSE
Customer Information:
  Name: Premium Customer
  Email: premium@example.com
  Phone: +1-555-9999
  Status: active

  Ticket History (1 tickets):
    - Account upgrade request (Status: open, Priority: medium)

General support query



### Scenario 3: Complex Query

In [30]:
print("\n" + "#"*70)
print("# SCENARIO 3: Complex Query")
print("#"*70)

result3 = run_query_with_agents("Show me all active customers who have open tickets")


######################################################################
# SCENARIO 3: Complex Query
######################################################################

 ROUTER AGENT: Show me all active customers who have open tickets

 A2A: Router → Customer Data Agent
   Customer Data Agent calling MCP: list_customers(status='active')
    Retrieved 25 active customers
   Customer Data Agent calling MCP: get_customer_history() for each
    Found 12 customers with open tickets

 Router synthesizing final response...

 FINAL RESPONSE
Found 12 active customers with open tickets:

  • John Doe (ID: 1)
    Email: john.doe@example.com
    Open Tickets: 2
      - Cannot login to account (Priority: high)
      - Cannot login to account (Priority: high)

  • Jane Smith (ID: 2)
    Email: jane.smith@example.com
    Open Tickets: 2
      - Feature request: dark mode (Priority: low)
      - Feature request: dark mode (Priority: low)

  • Alice Williams (ID: 4)
    Email: alice.w@techcorp.com
 

### Scenario 4: Escalation

In [31]:
print("\n" + "#"*70)
print("# SCENARIO 4: Escalation")
print("#"*70)

result4 = run_query_with_agents("I've been charged twice, please refund immediately!")


######################################################################
# SCENARIO 4: Escalation
######################################################################

 ROUTER AGENT: I've been charged twice, please refund immediately!

 A2A: Router → Support Agent
   Support Agent analyzing request...
    Identified: Urgent billing issue - ESCALATING

 Router synthesizing final response...

 FINAL RESPONSE

ESCALATION: Billing dispute detected



### Scenario 5: Multi-Intent

In [32]:
print("\n" + "#"*70)
print("# SCENARIO 5: Multi-Intent")
print("#"*70)

# Part A: Update email
print("\nPart A: Updating email for customer 1...")
update_result = test_mcp_tool("update_customer", {
    "customer_id": 1,
    "data": {"email": "newemail@test.com"}
})
print(f"✓ Email updated: {update_result}\n")

# Part B: Get history
print("Part B: Getting ticket history...")
result5 = run_query_with_agents("Show ticket history for customer 1")


######################################################################
# SCENARIO 5: Multi-Intent
######################################################################

Part A: Updating email for customer 1...
✓ Email updated: {'success': True, 'customer_id': 1, 'updated_fields': ['email']}

Part B: Getting ticket history...

 ROUTER AGENT: Show ticket history for customer 1

 A2A: Router → Customer Data Agent
   Customer Data Agent calling MCP: get_customer(1)
    Retrieved: John Doe
    Email: newemail@test.com
    Status: active
   Customer Data Agent calling MCP: get_customer_history(1)
    Found 4 tickets

 Router synthesizing final response...

 FINAL RESPONSE
Customer Information:
  Name: John Doe
  Email: newemail@test.com
  Phone: +1-555-0101
  Status: active

  Ticket History (4 tickets):
    - Cannot login to account (Status: open, Priority: high)
    - Password reset not working (Status: in_progress, Priority: medium)
    - Cannot login to account (Status: open, Priority:

---
# Conclusion

## What I Learned

Through this assignment, I gained hands-on experience implementing a multi-agent system with A2A coordination and MCP integration. The key learning was understanding how to properly integrate all three parts: the agents from Part 1 must use the MCP tools from Part 2, coordinated through Part 3's orchestration patterns. I learned that the Router Agent acts as an intelligent orchestrator, analyzing queries using pattern matching and routing them to specialized agents with clear separation of concerns. The MCP tools provide a clean interface between agent logic and data access, making the system modular and maintainable. I also learned about A2A communication patterns and how proper logging makes multi-agent interactions transparent and debuggable.

## Challenges Faced

The main challenge was adapting the reference notebook's A2A server pattern (which uses separate HTTP servers on different ports) to work in a Colab environment where persistent servers are difficult to maintain. I solved this by implementing the same logical coordination flow with direct function calls while preserving explicit A2A communication logging at each step. Another challenge was demonstrating clear A2A coordination for complex queries like "show all active customers with open tickets," which required the Customer Data Agent to make multiple MCP calls (list customers, then get history for each) and filter results before returning to the Router. Finally, balancing code simplicity for educational purposes while maintaining production-quality patterns like error handling, type hints, and comprehensive logging required careful design decisions.